In [1]:
%matplotlib

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import FloatSlider

def cartesian_product(*arrays):
    return np.array(np.meshgrid(*arrays)).T.reshape(-1, len(arrays))

def graph(x, fn=None, y=None, eqn=None, xlim=None, ylim=None, title=None, fn_params=None, fn_sliders=None):
    '''
    Can graph functions f(x), implicit equations f(x,y)=g(x,y),
    and approximate binary relations of {(x,y): f(x,y)==True}
    
    Graphing f(x) requires
        x: numpy.ndarray
        fn: function of x
        y=None
        eqn=None

    Graphing f(x,y)=g(x,y) requires
        x: numpy.ndarray
        y: numpy.ndarray
        eqn: {'lhs': f, 'rhs': g}

    Graphing binary relation requires
        x: numpy.ndarray
        y: numpy.ndarray
        fn: binary relation (boolean function) on x and y
        eqn=None
    
    fn_params (optional): dictionary of numpy.ndarrays
    fn_sliders (optional): list of dictionary parameters:
        {'name', 'min', 'max', 'value'}
    
    functions fn, eqn['lhs'], eqn['rhs'] should handle additional
    parameters if defined in fn_params and fn_sliders
    '''
    def init(ax):
        if xlim is not None: ax.set_xlim(xlim)
        if ylim is not None: ax.set_ylim(ylim)
        if title is not None: ax.set_title(title)
        ax.grid(True, which='both')
        # set the x-spine (see below for more info on `set_position`)
        ax.spines['left'].set_position('zero')
        # turn off the right spine/ticks
        ax.spines['right'].set_color('none')
        ax.yaxis.tick_left()
        # set the y-spine
        ax.spines['bottom'].set_position('zero')
        # turn off the top spine/ticks
        ax.spines['top'].set_color('none')
        ax.xaxis.tick_bottom()
    
    fig, ax = plt.subplots()
    init(ax)
    if y is not None:
        if eqn is None: domain = cartesian_product(x, y)
        else:
            X, Y = np.meshgrid(x, y)
            lhs, rhs = eqn['lhs'], eqn['rhs']
    param_list = []
    plots = []
    if fn_sliders is None: args = {}
    else: args = {slider['name']: slider['value'] for slider in fn_sliders}
    if fn_params is None:
        param_list.append({})
        if y is None: plot, = ax.plot(x, fn(x, **args), '-')
        else:
            if eqn is None:
                x_data, y_data = [], []
                for pt in domain:
                    if fn(x=pt[0], y=pt[1], **args):
                        x_data.append(pt[0])
                        y_data.append(pt[1])
                plot, = ax.plot(x_data, y_data, '.')
            else: plot = ax.contour(
                X, Y, lhs(X, Y, **args) - rhs(X, Y, **args), [0]
            )
        plots.append(plot)
    else:
        if len(fn_params) == 1:
            k = list(fn_params.keys())[0]
            param_list = [{k: v} for v in fn_params[k]]
        else:
            param_space = cartesian_product(*[v for k, v in fn_params.items()])
            for row in param_space:
                params = {}
                for k, v in zip(fn_params, row):
                    params[k] = v
                param_list.append(params)
        for params in param_list:
            label = ""
            for k, v in params.items():
                label += k + '=' + str(v) + ','
            if y is None: plot, = ax.plot(x, fn(x, **params, **args), '-', label=label)
            else:
                if eqn is None:
                    x_data, y_data = [], []
                    for pt in domain:
                        if fn(x=pt[0], y=pt[1], **params, **args):
                            x_data.append(pt[0])
                            y_data.append(pt[1])
                    plot, = ax.plot(x_data, y_data, '.', label=label)
                else: plot = ax.contour(
                    X, Y, lhs(X, Y, **params, **args)-rhs(X, Y, **params, **args), [0]
                )
            plots.append(plot)
        ax.legend()

    if fn_sliders is not None:
        def update(**args):
            if y is None:
                for i, params in enumerate(param_list):
                    plots[i].set_ydata(fn(x, **params, **args))
            elif eqn is None:
                for i, params in enumerate(param_list):
                    x_data, y_data = [], []
                    for pt in domain:
                        if fn(x=pt[0], y=pt[1], **params, **args):
                            x_data.append(pt[0])
                            y_data.append(pt[1])
                    plots[i].set_xdata(x_data)
                    plots[i].set_ydata(y_data)
            else:
                for i, params in enumerate(param_list):
                    ax.clear()
                    init(ax)
                    ax.contour(
                        X, Y,
                        lhs(X, Y, **params, **args)-rhs(X, Y, **params, **args),
                        [0]
                    )
            fig.canvas.draw()

        sliders = {slider['name']: FloatSlider(
            min=slider['min'], max=slider['max'], value=slider['value'],
            step=0.01*(slider['max']-slider['min']), continuous_update=False
        ) for slider in fn_sliders}
        interact(update, **sliders)

Using matplotlib backend: MacOSX


In [2]:
graph(
    x=np.linspace(-3, 3, 100),
    y=np.linspace(-3, 3, 100),
    fn=lambda x, y, a, b: np.isclose(y**2, x**3+a*x+b, rtol=0.2),
    xlim=(-3, 3), ylim=(-3, 3),
    title=r'$y^2=x^3+ax+b$',
    fn_params={'b': np.linspace(-5, 5, 5)},
    fn_sliders=[
        {'name': 'a', 'min': -5, 'max': 5, 'value': -2},
    ]
)

interactive(children=(FloatSlider(value=-2.0, continuous_update=False, description='a', max=5.0, min=-5.0), Ou…

In [3]:
graph(
    x=np.linspace(-3, 3, 100),
    y=np.linspace(-3, 3, 100),
    eqn={
        'lhs': lambda x, y, a, b: y**2,
        'rhs': lambda x, y, a, b: x**3+a*x+b
    },
    xlim=(-3, 3), ylim=(-3, 3),
    title=r'$y^2=x^3+ax+b$',
    #fn_params={'b': np.linspace(-3, 3, 3)}, broken
    fn_sliders=[
        {'name': 'a', 'min': -5, 'max': 5, 'value': -2},
        {'name': 'b', 'min': -5, 'max': 5, 'value': 1}
    ]
)

interactive(children=(FloatSlider(value=-2.0, continuous_update=False, description='a', max=5.0, min=-5.0), Fl…

In [4]:
graph(
    x=np.linspace(-10, 10, 1000),
    fn=lambda x, a, b, c: a*x**2+b*x+c,
    xlim=(-7, 7), ylim=(-7, 7),
    title=r'$y(x)=ax^2+bx+c$',
    fn_params={
        'a': np.linspace(-1, 1, 3),
        'c': np.linspace(-2, 2, 3),
    },
    fn_sliders=[{'name': 'b', 'min': -2, 'max': 2, 'value': 1}]
)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='b', max=2.0, min=-2.0, step…

In [5]:
graph(
    x=np.linspace(-10, 10, 1000),
    fn=lambda x, C, k, alpha, omega: C*np.exp(-k*x)+k*alpha/(omega**2+k**2)*(-omega*np.cos(omega*x)+k*np.sin(omega*x)),
    xlim=(-7, 7), ylim=(-7, 7),
    title=r'$y(t)=Ce^{-kt}+\frac{k\alpha}{\omega^2+k^2}(-\omega\cos{\omega t}+k\sin{\omega t})$',
    fn_params={'C': np.linspace(-5, 5, 5)},
    fn_sliders=[
        {'name': 'k', 'min': -2, 'max': 2, 'value': 1},
        {'name': 'alpha', 'min': -2, 'max': 2, 'value': 1},
        {'name': 'omega', 'min': -2, 'max': 2, 'value': 1}
    ]
)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='k', max=2.0, min=-2.0, step…

In [6]:
graph(
    x=np.linspace(-10, 10, 1000),
    fn=lambda x, C: (2*x+C) * np.exp(-(x**2)),
    xlim=(-5, 5), ylim=(-5, 5),
    title=r'$y(x)=e^{-x^2}(2x+C)$',
    fn_params={'C': np.linspace(-5, 5, 5)}
)

In [7]:
graph(
    x=np.linspace(-10, 10, 1000),
    fn=lambda x, C: C*np.exp(-3*x)+x-1/3+2/5*np.exp(2*x)+4/13*(3*np.cos(2*x)+2*np.sin(2*x)),
    xlim=(-5, 5), ylim=(-10, 10),
    title=r'$y(x)=Ce^{-3x}+x-\frac{1}{3}+\frac{2}{5}e^{2x}+\frac{4}{13}(3\cos{2x}+2\sin{2x})$',
    fn_params={'C': np.linspace(-5, 5, 5)}
)